In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np


In [4]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
df = pd.read_csv('/content/drive/MyDrive/StumbleUponEvergreen/train.tsv',sep = '\t', header = 0)
df.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0,0.095861,0,0.265656,0.035343,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,0.446143,0.0,0,0.024908,0,0.228887,0.050473,1,1,14,0,12032,162,10,0.098765,0.082569,0


In [6]:
boilerplates =list(df['boilerplate'].astype(str))
labels = list(df.label)
print(labels[0])
boilerplates[0]


0


'{"title":"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries","body":"A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s largest provider of computer services looks to Silicon Valley for input gleaning many ideas from its Almaden research center in San Jose

In [7]:
vocab_size = 2000
embedding_dim = 16
max_length = 1500
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 5916 #80 percent of the dataset.
validation_size = 7025 #15 percent of the dataset.

In [9]:
training_sentences = boilerplates[0:training_size]
validation_sentences = boilerplates[training_size:validation_size]
testing_sentences = boilerplates[validation_size:]
training_labels = labels[0:training_size]
validation_labels = labels[training_size:validation_size]
testing_labels = labels[validation_size:]

In [10]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

In [11]:
word_index = tokenizer.word_index


In [12]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [13]:
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [14]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1500, 16)          32000     
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                12544     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 46,129
Trainable params: 46,129
Non-trainable params: 0
_________________________________________________________________


In [17]:
with tf.device('/device:GPU:0'):
  num_epochs = 50
  training_padded = np.array(training_padded)
  training_labels = np.array(training_labels)
  validation_padded = np.array(validation_padded)
  validation_labels = np.array(validation_labels)
  history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(validation_padded, validation_labels), verbose=1)


Epoch 1/50
185/185 [==============================] - 19s 84ms/step - loss: 0.6709 - accuracy: 0.5681 - val_loss: 0.4921 - val_accuracy: 0.7773
Epoch 2/50
185/185 [==============================] - 15s 81ms/step - loss: 0.4462 - accuracy: 0.8070 - val_loss: 0.4785 - val_accuracy: 0.7899
Epoch 3/50
185/185 [==============================] - 15s 80ms/step - loss: 0.4156 - accuracy: 0.8219 - val_loss: 0.4742 - val_accuracy: 0.7836
Epoch 4/50
185/185 [==============================] - 15s 81ms/step - loss: 0.3624 - accuracy: 0.8567 - val_loss: 0.4839 - val_accuracy: 0.7818
Epoch 5/50
185/185 [==============================] - 15s 80ms/step - loss: 0.3590 - accuracy: 0.8572 - val_loss: 0.5113 - val_accuracy: 0.7800
Epoch 6/50
185/185 [==============================] - 15s 80ms/step - loss: 0.3269 - accuracy: 0.8719 - val_loss: 0.5035 - val_accuracy: 0.7656
Epoch 7/50
185/185 [==============================] - 15s 81ms/step - loss: 0.3217 - accuracy: 0.8776 - val_loss: 0.5178 - val_accuracy:

In [19]:
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)
loss, accuracy = model.evaluate(testing_padded, testing_labels)

12/12 [==============================] - 1s 45ms/step - loss: 1.3299 - accuracy: 0.7486


In [20]:
model.save('/content/drive/MyDrive/StumbleUponEvergreen/model.h5')

In [21]:
df2 = pd.read_csv('/content/drive/MyDrive/StumbleUponEvergreen/test.tsv',sep = '\t', header = 0)
df2.head()


,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio
0,http://www.lynnskitchenadventures.com/2009/04/...,5865,"{""title"":""Homemade Enchilada Sauce Lynn s Kitc...",recreation,0.443906,2.558140,0.389706,0.257353,0.044118,0.022059,0.489572,0.0,0,0.067143,0,0.230285,0.199438,1,1,15,0,5643,136,3,0.242647,0.080597
1,http://lolpics.se/18552-stun-grenade-ar,782,"{""title"":""lolpics Stun grenade ar "",""body"":"" f...",culture_politics,0.135844,3.771429,0.461538,0.205128,0.051282,0.000000,0.782051,0.0,0,0.042857,0,0.365962,0.080000,?,1,62,0,382,39,2,0.128205,0.176471
2,http://www.xcelerationfitness.com/treadmills.html,6962,"{""title"":""Treadmills "",""body"":"" treadmills, st...",?,?,2.269565,0.495726,0.384615,0.170940,0.170940,1.250000,0.0,0,0.058824,0,0.161901,10.000000,?,1,42,0,2420,117,1,0.581197,0.125000
3,http://www.bloomberg.com/news/2012-02-06/syria...,7640,"{""title"":""Father s Tactics Used by Assad to Cr...",culture_politics,0.90259,2.523490,0.705502,0.346278,0.122977,0.090615,0.449366,0.0,0,0.058081,0,0.146593,0.005964,1,1,41,0,5559,309,10,0.038835,0.063126
4,http://www.wired.com/gadgetlab/2011/12/stem-tu...,3589,"{""title"":""Stem Turns Lemons and Limes Into Jui...",science_technology,0.486363,1.848000,0.470968,0.161290,0.032258,0.000000,0.453757,0.0,0,0.093023,0,0.244141,0.035714,1,0,34,0,2209,155,10,0.096774,0.065341


In [23]:
prediction_sentences = list(df2['boilerplate'].astype(str))
print(prediction_sentences[0])

{"title":"Homemade Enchilada Sauce Lynn s Kitchen Adventures ","body":"I usually buy my enchilada sauce Yes I knew I should be making it but I had never found a recipe that I was really happy with I had tried several and they just weren t very good So I stuck to the canned stuff you can get at the grocery store I was recently talking to a friend of mine about this She lived in Mexico for a few years so she knows some about Mexican cooking I asked her how she made her enchilada sauce She told me the basics and then gave me an exact recipe I decided to give it a try This recipe was really good This was the best enchilada sauce that I have made It had great flavor I think it was even better than the canned sauce My husband thought it could have been spicier But he likes his enchiladas spicy You can always add more chili powder or chilies if you like it really spicy The kids and I thought it was really good just like it is I did change two things It called for green onions I did not have a

In [24]:
prediction_sequences = tokenizer.texts_to_sequences(prediction_sentences)
prediction_padded = pad_sequences(prediction_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [25]:
prediction = model.predict(prediction_padded, verbose=0)

In [28]:
prediction = np.around(prediction)
print(prediction)

[[1.]
 [0.]
 [1.]
 ...
 [1.]
 [1.]
 [0.]]


In [61]:
pred_df = pd.DataFrame(prediction, columns=['label'])
url_df = pd.DataFrame(data = df2['urlid'], columns=['urlid'])
url_df.reset_index(drop = True, inplace = True)
pred_df.reset_index(drop = True, inplace = True)
len(pred_df)


3171

In [62]:
url_df.head()

,urlid
0,5865
1,782
2,6962
3,7640
4,3589


In [63]:
submission_df = pd.concat([url_df, pred_df], axis=1)
submission_df.head()

,urlid,label
0,5865,1.0
1,782,0.0
2,6962,1.0
3,7640,0.0
4,3589,0.0


In [64]:
df.to_csv('/content/drive/MyDrive/StumbleUponEvergreen/sampleSubmission.csv')